In [30]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=16, app_name='west0_mclaren_workshop_use', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 16 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_mclaren_workshop_use...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [29]:
sphynx.stop()

In [3]:
def get_workshop_rate(start_date, end_date):
    bp_purchase = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where((col("Currency").like("%bp%")) & (col("AnalyticEventType") == "workshop")) 
    
    workshop_gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name")=="202207_workshop")
    mclaren_gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name")=="202209_season_workshop")
    
    open_mclaren_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date).where(col("ItemDescId").isin(["itemdesc.14300003", "itemdesc.14300004", "itemdesc.14300005"]))
    open_normal_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date).where(~col("ItemDescId").isin(["itemdesc.14300003", "itemdesc.14300004", "itemdesc.14300005"]))
    
    mclaren_craft_items = mysql.read_table(spark, "labs", "item_meta_brodesc").where(col("item_name").like("%McLaren%")).withColumn('item_id_number', split(col("item_id"), '\.').getItem(1))
    craft = load_schema.lobby(spark, "pc", "live", "WorkshopCrafted", start_date, end_date).withColumn('item_id_number', split(col("ItemDescId"), '\.').getItem(1))
    mclaren_craft = craft.join(mclaren_craft_items, "item_id_number")
    normal_craft = craft.join(mclaren_craft_items, "item_id_number", "leftanti")
    
    disassemble = load_schema.lobby(spark, "pc", "live", "WorkshopDisassembled", start_date, end_date)
    repurpose = load_schema.lobby(spark, "pc", "live", "WorkshopRepurposed", start_date, end_date)
    
    special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where(col("Currency") == "artisanstoken")
    mclaren_special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where(col("Currency").like("%seasonaltoken%"))
    
    workshop_user = (bp_purchase.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("bp"))) \
        .unionByName(workshop_gcoin.select("account_id").distinct().withColumn("action", lit("workshop_gcoin"))) \
        .unionByName(mclaren_gcoin.select("account_id").distinct().withColumn("action", lit("mclaren_gcoin"))) \
        .unionByName(open_mclaren_crate.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("mclaren_crate_open"))) \
        .unionByName(open_normal_crate.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("normal_crate_open"))) \
        .unionByName(mclaren_craft.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("mclaren_craft"))) \
        .unionByName(normal_craft.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("normal_craft"))) \
        .unionByName(disassemble.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("disassemble"))) \
        .unionByName(repurpose.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("repurpose"))) \
        .unionByName(special_craft.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("special_craft"))) \
        .unionByName(mclaren_special_craft.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("mclaren_special_craft")))
    
    return workshop_user

In [4]:
workshop_use = get_workshop_rate("2022-09-07", "2022-11-08")

In [33]:
df = mysql.read_table(spark, "labs", "mclaren_seg_pre_and_post_info")

In [34]:
user_group_info = df.select("account_id", "group", "mclaren_group")

In [7]:
mclaren_user_workshop_use = user_group_info.join(workshop_use, "account_id", "left")

In [8]:
mysql.drop_table("labs", "mclaren_seg_workshop_use")

In [9]:
mysql.insert_table(mclaren_user_workshop_use, "labs", "mclaren_seg_workshop_use")

In [31]:
start_date = "2022-09-07"
end_date = "2022-11-08"
mclaren_craft_items = mysql.read_table(spark, "labs", "item_meta_brodesc").where(col("item_name").like("%McLaren%")).withColumn('item_id_number', split(col("item_id"), '\.').getItem(1))
craft = load_schema.lobby(spark, "pc", "live", "WorkshopCrafted", start_date, end_date).withColumn('item_id_number', split(col("ItemDescId"), '\.').getItem(1))
mclaren_craft = craft.join(mclaren_craft_items, "item_id_number")
normal_craft = craft.join(mclaren_craft_items, "item_id_number", "leftanti")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33915)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


In [15]:
mclaren_craft.select(countDistinct("AccountId"), count("*")).show(truncate=False)

+-------------------------+--------+
|count(DISTINCT AccountId)|count(1)|
+-------------------------+--------+
|450690                   |927447  |
+-------------------------+--------+



In [32]:
mclaren_craft = mclaren_craft.withColumnRenamed("AccountId", "account_id")

In [35]:
mclaren_craft = mclaren_craft.join(user_group_info, "account_id")

In [36]:
mclaren_craft_by_group = mclaren_craft.groupBy("group", "mclaren_group", "item_name").agg(count("*").alias("craft_cnt"))

In [39]:
mclaren_craft_by_group.printSchema()

root
 |-- group: string (nullable = true)
 |-- mclaren_group: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- craft_cnt: long (nullable = false)



In [40]:
from pyspark.sql.window import Window
Window_Spec  = Window.partitionBy(["group", "mclaren_group"]).orderBy(col("craft_cnt").desc())
craft_rank_by_group = mclaren_craft_by_group.withColumn("craft_rank", row_number().over(Window_Spec))

In [41]:
craft_rank_by_group.toPandas().to_csv("./mclaren_craft_by_group.csv", index=False)

In [44]:
mclaren_craft_only_by_group = mclaren_craft.groupBy("group","item_name").agg(count("*").alias("craft_cnt"))

In [45]:
Window_Spec  = Window.partitionBy(["group"]).orderBy(col("craft_cnt").desc())
craft_rank_only_by_group = mclaren_craft_only_by_group.withColumn("craft_rank", row_number().over(Window_Spec))

In [46]:
craft_rank_only_by_group.toPandas().to_csv("./mclaren_craft_only_by_group.csv", index=False)

In [48]:
mclaren_craft.groupBy("group","item_name").agg(count("*").alias("craft_cnt"), countDistinct("account_id").alias("user_cnt")).toPandas().sort_values(["group", "craft_cnt"])

,group,item_name,craft_cnt,user_cnt
31,M-1,McLAREN SET 1,3430,3429
24,M-1,McLAREN SET 2,3666,3666
28,M-1,"""McLaren GT Standard - Onyx Black"" Coupe RB",13546,13484
22,M-1,McLaren - Helmet (Level 2),28239,28012
26,M-1,McLaren Backpack (Level 2),29085,28885
19,M-1,McLaren Backpack (Level 3),29371,29128
33,M-1,McLaren Parachute,29982,29794
9,M-1,McLaren - Helmet (Level 3),30069,29839
29,M-2,McLAREN SET 2,2674,2667
8,M-2,McLAREN SET 1,2743,2736


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 44332)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp